In [7]:
import os
import json
import librosa
import pandas as pd
import numpy as np
import soundfile as sf
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split

In [5]:
DATA_PATH = '../data/spcup_2022_training_part1'
IMAGE_PATH = '../data/images'
Labeled_dir = '../data/spcup_2022_training_part1/labels.csv'
desc_path = '../data/train_data.csv'

In [4]:
label_df = pd.read_csv(Labeled_dir)
label_df.head()

,track,algorithm
0,00050dd7458cf08e594c797930696bce.wav,4
1,00070e7c531000d3dddc735d107275a9.wav,2
2,000f0711027a69b7f3886c2dbcb7d41f.wav,3
3,001e28e66dee24408aaf3480dfb95fbe.wav,1
4,001eee950f60613869544b72cd48fe97.wav,2


In [5]:
label_df['wav_path'] = label_df['track'].map(lambda x: DATA_PATH+'/'+x)
label_df.head()

,track,algorithm,wav_path
0,00050dd7458cf08e594c797930696bce.wav,4,../data/spcup_2022_training_part1/00050dd7458c...
1,00070e7c531000d3dddc735d107275a9.wav,2,../data/spcup_2022_training_part1/00070e7c5310...
2,000f0711027a69b7f3886c2dbcb7d41f.wav,3,../data/spcup_2022_training_part1/000f0711027a...
3,001e28e66dee24408aaf3480dfb95fbe.wav,1,../data/spcup_2022_training_part1/001e28e66dee...
4,001eee950f60613869544b72cd48fe97.wav,2,../data/spcup_2022_training_part1/001eee950f60...


In [6]:
spect_type = 'mel'
def get_image(wav_path):
    y, sr = librosa.load(wav_path)
    if spect_type == 'stft':
        D = librosa.stft(y)  # STFT of y
        spec = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    elif spect_type == 'mel':
        M = librosa.feature.melspectrogram(y=y, sr=sr)
        spec = librosa.power_to_db(M, ref=np.max)
    elif spect_type == 'chroma':
        spec = librosa.feature.chroma_cqt(y=y, sr=sr)

    return spec

In [7]:
label_df['image_path'] = label_df['wav_path'].map(lambda x: get_image(x))
label_df.head()

,track,algorithm,wav_path,image_path
0,00050dd7458cf08e594c797930696bce.wav,4,../data/spcup_2022_training_part1/00050dd7458c...,"[[-77.97527, -63.775597, -59.59822, -54.112976..."
1,00070e7c531000d3dddc735d107275a9.wav,2,../data/spcup_2022_training_part1/00070e7c5310...,"[[-62.508484, -54.88949, -57.50805, -61.510017..."
2,000f0711027a69b7f3886c2dbcb7d41f.wav,3,../data/spcup_2022_training_part1/000f0711027a...,"[[-66.01921, -66.485886, -64.43065, -61.471973..."
3,001e28e66dee24408aaf3480dfb95fbe.wav,1,../data/spcup_2022_training_part1/001e28e66dee...,"[[-65.137115, -61.9981, -62.22146, -67.04489, ..."
4,001eee950f60613869544b72cd48fe97.wav,2,../data/spcup_2022_training_part1/001eee950f60...,"[[-68.83245, -59.767883, -55.963326, -54.93239..."


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X, y = label_df['image_path'].values, label_df['algorithm'].values
# stratified split dataset into train-validation
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1)

In [ ]:
for i in X_train[:100]:
    print(i.shape)

In [12]:
a = np.arange(100).reshape((10,10))
a.take([8,9,10], mode="wrap", axis=1).take([9,10,11], mode="wrap", axis=0)

array([[98, 99, 90],
       [ 8,  9,  0],
       [18, 19, 10]])

In [15]:
result = np.zeros([12, 13])
result[:a.shape[0],:a.shape[1]] = a

In [16]:
result

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.,  0.,  0.,  0.],
       [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.,  0.,  0.,  0.],
       [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.,  0.,  0.,  0.],
       [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,  0.,  0.,  0.],
       [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.,  0.,  0.,  0.],
       [50., 51., 52., 53., 54., 55., 56., 57., 58., 59.,  0.,  0.,  0.],
       [60., 61., 62., 63., 64., 65., 66., 67., 68., 69.,  0.,  0.,  0.],
       [70., 71., 72., 73., 74., 75., 76., 77., 78., 79.,  0.,  0.,  0.],
       [80., 81., 82., 83., 84., 85., 86., 87., 88., 89.,  0.,  0.,  0.],
       [90., 91., 92., 93., 94., 95., 96., 97., 98., 99.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [19]:
def crop_with_padd(image, size=(128,128)):
    image = image[:size[0], :size[1]]
    result = np.zeros(size)
    result[:image.shape[0], :image.shape[1]] = image
    return result

In [20]:
X_train = np.array([crop_with_padd(i) for i in X_train])
X_test = np.array([crop_with_padd(i) for i in X_test])

***

In [3]:
desc_df = pd.read_csv('../data/train_data.csv')
desc_df.head()

,audio_id,label,wav,image
0,00050dd7458cf08e594c797930696bce,4,../data/spcup_2022_training_part1/00050dd7458c...,../data/images/00050dd7458cf08e594c797930696bc...
1,00070e7c531000d3dddc735d107275a9,2,../data/spcup_2022_training_part1/00070e7c5310...,../data/images/00070e7c531000d3dddc735d107275a...
2,000f0711027a69b7f3886c2dbcb7d41f,3,../data/spcup_2022_training_part1/000f0711027a...,../data/images/000f0711027a69b7f3886c2dbcb7d41...
3,001e28e66dee24408aaf3480dfb95fbe,1,../data/spcup_2022_training_part1/001e28e66dee...,../data/images/001e28e66dee24408aaf3480dfb95fb...
4,001eee950f60613869544b72cd48fe97,2,../data/spcup_2022_training_part1/001eee950f60...,../data/images/001eee950f60613869544b72cd48fe9...


In [4]:
import cv2

In [5]:
desc_df['image_np'] = desc_df['image'].map(lambda x: cv2.imread(x))
desc_df.head()

,audio_id,label,wav,image,image_np
0,00050dd7458cf08e594c797930696bce,4,../data/spcup_2022_training_part1/00050dd7458c...,../data/images/00050dd7458cf08e594c797930696bc...,"[[[149, 149, 149], [122, 122, 122], [163, 163,..."
1,00070e7c531000d3dddc735d107275a9,2,../data/spcup_2022_training_part1/00070e7c5310...,../data/images/00070e7c531000d3dddc735d107275a...,"[[[42, 42, 42], [73, 73, 73], [184, 184, 184],..."
2,000f0711027a69b7f3886c2dbcb7d41f,3,../data/spcup_2022_training_part1/000f0711027a...,../data/images/000f0711027a69b7f3886c2dbcb7d41...,"[[[107, 107, 107], [145, 145, 145], [211, 211,..."
3,001e28e66dee24408aaf3480dfb95fbe,1,../data/spcup_2022_training_part1/001e28e66dee...,../data/images/001e28e66dee24408aaf3480dfb95fb...,"[[[48, 48, 48], [241, 241, 241], [6, 6, 6], [5..."
4,001eee950f60613869544b72cd48fe97,2,../data/spcup_2022_training_part1/001eee950f60...,../data/images/001eee950f60613869544b72cd48fe9...,"[[[83, 83, 83], [169, 169, 169], [66, 66, 66],..."


In [6]:
desc_df['image_np'].values[0].shape

(128, 256, 3)

In [ ]:
def load_from_images(desc_path, split_ratio=0.1):
    """load from saved image folder. i.e. the image save has to done under "json_creator.ipynb" file
       * takes nearly 2 minutes

    Args:
        desc_path ([type]): summary .csv file path
        split_ratio (float, optional): [test set size]. Defaults to 0.1.

    Returns:
        [4 np.ndarrays]: [X_train, X_test, y_train, y_test]
    """
    desc_df = pd.read_csv(desc_path)
    desc_df['image'] = desc_df['image'].map(lambda x: cv2.imread(x))
    X, y = label_df['image'].values, label_df['algorithm'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=split_ratio)
    return X_train, X_test, y_train, y_test

***

In [13]:
class SpecLoader:

    def get_image(self, wav_path):
        y, sr = librosa.load(wav_path)
        if self.spect_type == 'stft':
            D = librosa.stft(y)  # STFT of y
            spec = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        elif self.spect_type == 'mel':
            M = librosa.feature.melspectrogram(y=y, sr=sr)
            spec = librosa.power_to_db(M, ref=np.max)
        elif self.spect_type == 'chroma':
            spec = librosa.feature.chroma_cqt(y=y, sr=sr)

        return spec

    def crop_with_padd(self, image):
        size = self.shape
        image = image[:size[0], :size[1]]
        result = np.zeros(size)
        result[:image.shape[0], :image.shape[1]] = image
        return result

    def load_from_df(self, label_path, data_path, split_ratio=0.1):
        """load images directly from the "data_summary.csv" file 
        * takes nearly 8 minutes

        Args:
            label_path ([type]): [.csv file path]
            data_path ([type]): [.wav files folder directory]
            split_ratio (float, optional): [test set size]. Defaults to 0.1.

        Returns:
            [4 np.ndarrays]: [X_train, X_test, y_train, y_test]
        """
        label_df = pd.read_csv(label_path)
        label_df['wav_path'] = label_df['track'].map(lambda x: data_path+'/'+x)

        label_df['image'] = label_df['wav_path'].map(lambda x: self.get_image(x))
        X, y = label_df['image'].values, label_df['algorithm'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=split_ratio)
        return X_train, X_test, y_train, y_test

    def load_from_images(self, desc_path, split_ratio=0.1):
        """load from saved image folder. i.e. the image save has to done under "json_creator.ipynb" file
        * takes nearly 2 minutes

        Args:
            desc_path ([type]): summary .csv file path
            split_ratio (float, optional): [test set size]. Defaults to 0.1.

        Returns:
            [4 np.ndarrays]: [X_train, X_test, y_train, y_test]
        """
        desc_df = pd.read_csv(desc_path)
        desc_df['image'] = desc_df['image'].map(lambda x: cv2.imread(x, cv2.IMREAD_GRAYSCALE))
        X, y = desc_df['image'].values, desc_df['label'].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=split_ratio)
        return X_train, X_test, y_train, y_test

    def load(self, label_path, data_path, mode='image', split_ratio=0.1, spect='mel', shape=(128,128)):
        self.spect_type = spect
        self.shape = shape
        if mode == 'image':
            X_train, X_test, y_train, y_test = self.load_from_images(label_path, split_ratio)
        elif mode == 'df':
            X_train, X_test, y_train, y_test = self.load_from_df(label_path, data_path, split_ratio)

        X_train = np.array([self.crop_with_padd(i) for i in X_train])
        X_test = np.array([self.crop_with_padd(i) for i in X_test])

        return (X_train, y_train), (X_test, y_test)

In [14]:
dloader = SpecLoader()
(X_train, y_train), (X_test, y_test) = dloader.load(desc_path,IMAGE_PATH, mode='image' )

In [16]:
X_train.shape

(4500, 128, 128)